In [31]:
import pandas as pd
import math
import json
import numpy as np

# READING FILES

In [32]:
### THIS IS USED TO COMBINE SMALL AREAS:

# # Read lookup for geographic hierarchy
# tiers = pd.read_csv('./LAD_County_Region_Country_2021.csv')

# # Combine small areas
# tiers = tiers.append({'LAD21CD': 'E09000001/E09000033', 'LAD21NM': 'City of London/Westminster', 'CTY21CD': 'E13000001', 'CTY21NM': 'Inner London', 'RGN21CD': 'E12000007', 'RGN21NM': 'London', 'CTRY21CD': 'E92000001', 'CTRY21NM': 'England'}, ignore_index=True)

# tiers = tiers.append({'LAD21CD': 'E06000052/E06000053', 'LAD21NM': 'Cornwall/Isles of Scilly', 'CTY21CD': 'E06000052/E06000053', 'CTY21NM': 'Cornwall/Isles of Scilly', 'RGN21CD': 'E12000009', 'RGN21NM': 'South West', 'CTRY21CD': 'E92000001', 'CTRY21NM': 'England'}, ignore_index=True)

# # Remove the small areas (uncombined)
# tiers = tiers[tiers['LAD21NM'].apply(lambda x: x not in ['Cornwall', 'Isles of Scilly', 'Westminster', 'City of London'])]

# tiers

In [33]:
# Read lookup for geographic hierarchy
tiers = pd.read_csv('./LAD_County_Region_Country_2021.csv')

tiers

,LAD21CD,LAD21NM,CTY21CD,CTY21NM,RGN21CD,RGN21NM,CTRY21CD,CTRY21NM
0,E06000001,Hartlepool,E06000001,Hartlepool,E12000001,North East,E92000001,England
1,E06000002,Middlesbrough,E06000002,Middlesbrough,E12000001,North East,E92000001,England
2,E06000003,Redcar and Cleveland,E06000003,Redcar and Cleveland,E12000001,North East,E92000001,England
3,E06000004,Stockton-on-Tees,E06000004,Stockton-on-Tees,E12000001,North East,E92000001,England
4,E06000005,Darlington,E06000005,Darlington,E12000001,North East,E92000001,England
...,...,...,...,...,...,...,...,...
326,W06000020,Torfaen,W06000020,Torfaen,W92000004,Wales,W92000004,Wales
327,W06000021,Monmouthshire,W06000021,Monmouthshire,W92000004,Wales,W92000004,Wales
328,W06000022,Newport,W06000022,Newport,W92000004,Wales,W92000004,Wales
329,W06000023,Powys,W06000023,Powys,W92000004,Wales,W92000004,Wales


In [34]:
# Open data files containing nearest neighbour information and process into dictionaries
with open('ladsneighbours.json') as json_file:
    lads_neigh_raw = json.load(json_file)
lads_neigh = {}
for i in lads_neigh_raw:
    if (lads_neigh_raw[i][0]):
        lads_neigh[i] = lads_neigh_raw[i][0][0]
    else:
        lads_neigh[i] = lads_neigh_raw[i][1]

# # Add nearest neighbour for combined small areas Cornwall and Isles of Scilly and CoL/Westminster
# lads_neigh['E06000052/E06000053'] = lads_neigh['E06000052']
# lads_neigh['E09000001/E09000033'] = lads_neigh['E09000033']

countyneighbourdf = pd.read_csv('countyboundaries.csv')
county_neigh = {}
for i in countyneighbourdf.index:
    countycodelist = countyneighbourdf.iloc[i]['neighbours'].split(",")
    countylengthlist = countyneighbourdf.iloc[i]['lengths'].split(",")
    zipped = list(zip(countycodelist, countylengthlist))
    zipped = [i for i in zipped if i[0][:3] != "E08"]
    zipped = sorted(zipped, reverse=True, key= lambda x: int(x[1]))
    county_neigh[countyneighbourdf.iloc[i]['CTYUA21CD']] = zipped[0][0]

# Add nearest neighbours for met counties as they weren't included in boudary data
county_neigh['E11000001'] = 'E10000017'
county_neigh['E11000002'] = 'E10000017'
county_neigh['E11000003'] = 'E10000007'
county_neigh['E11000007'] = 'E06000057'
county_neigh['E11000005'] = 'E10000031'
county_neigh['E11000006'] = 'E10000023'
county_neigh['E13000001'] = 'E13000002'
county_neigh['E13000002'] = 'E13000001'

# # Add nearest neighbour for combined Cornwall and Isles of Scilly
# county_neigh['E06000052/E06000053'] = county_neigh['E06000052']

In [35]:
# Open and process data on statistical similarity
similars = {}
similar_raw = pd.read_csv('./LAD_MAY_2021_EW_Adjacency_Matrix.csv')
for i in similar_raw.index:
    similars[similar_raw.iloc[i]['LAD21CD']] = [similar_raw.iloc[i]['LAD1CD'],similar_raw.iloc[i]['LAD2CD'],similar_raw.iloc[i]['LAD3CD'],similar_raw.iloc[i]['LAD4CD']]

### THIS IS FOR COMBINED SMALL AREAS
# similars['E06000052/E06000053'] = similars['E06000052']
# similars['E09000001/E09000033'] = similars['E09000033']

# CREATE THE EMPTY OBJECTS FOR EACH PLACE

In [36]:
# A function to find a statistically similar area
def get_similar(code):
    # Create a list of similar areas
    theseSimilars = similars[code]
    # Get rid of NaNs for the list of similars
    theseSimilars = [i for i in theseSimilars if i == i]

    # Filter out codes for Scotland and NI
    theseSimilars = [i for i in theseSimilars if i[0] not in ['S', 'N']]

    # Remove small areas
    theseSimilars = [i for i in theseSimilars if i not in ['E09000001', 'E09000033', 'E06000053', 'E06000052']]

    # Create variable for most similar area
    if len(theseSimilars) > 0:
        similar = theseSimilars[0]
    else:
        similar = None

    # If the most similar area is also the nearest area, then chose another area
    if similar == las[nameLA]['near']:
        # If there's at least two similar areas reselect area
        if len(theseSimilars)>1:
            similar = theseSimilars[1]
        else:
            similar = None

    return similar

In [37]:
# Create and object to populate with each area's data
las = {}
counties = {}
regions = {}
countries = {}

# Iterate throough index of LA df
for i in tiers.index:

    # Create a variable which is an iterated row of the df
    row = tiers.loc[i]

    # The name of the area attached to this row
    nameLA = row['LAD21NM']
    nameCounty = row['CTY21NM']
    nameRegion = row['RGN21NM']
    nameCountry = row['CTRY21NM']

    # Add an object with name and code info and empty nested oobject for data
    las[nameLA] = {}
    las[nameLA]['name'] = nameLA
    las[nameLA]['parent'] = {
        'name': nameRegion,
        'code': row['RGN21CD']
    }
    las[nameLA]['country'] = nameCountry
    las[nameLA]['code'] = row['LAD21CD']
    las[nameLA]['type'] = 'LAD'
    las[nameLA]['data'] = {}

    counties[nameCounty] = {}
    counties[nameCounty]['name'] = nameCounty
    counties[nameCounty]['parent'] = {
        'name': nameRegion,
        'code': row['RGN21CD']
    }
    counties[nameCounty]['country'] = nameCountry
    counties[nameCounty]['code'] = row['CTY21CD']
    counties[nameCounty]['near'] = county_neigh[row['CTY21CD']]
    counties[nameCounty]['type'] = 'County'
    counties[nameCounty]['data'] = {}

    regions[nameRegion] = {}
    regions[nameRegion]['name'] = nameRegion
    regions[nameRegion]['parent'] = {
        'name': nameCountry,
        'code': row['CTRY21CD']
    }
    regions[nameRegion]['country'] = nameCountry
    regions[nameRegion]['code'] = row['RGN21CD']
    regions[nameRegion]['type'] = 'Region'
    regions[nameRegion]['data'] = {}

    countries[nameCountry] = {}
    countries[nameCountry]['name'] = nameCountry
    countries[nameCountry]['code'] = row['CTRY21CD']
    countries[nameCountry]['type'] = 'Country'
    countries[nameCountry]['data'] = {}

# DATA PROCESSING BEGINS BELOW

In [38]:
# LIST OF ALL TOPICS AND BOTH YEARS
topics = ['care',
 'religion',
 'ethnicity',
 'health',
 'economic',
 'household',
 'marital',
 'hours',
 'tenure',
 'disability',
 'national',
 'welsh',
 'population',
 'density',
 'age10yr',
 'agemed']
years = ['2011', '2021']

In [39]:
###  FUNCTION NOT NEEDED UNLESS MERGING SMALL AREAS

# def merge_small(df):
#     cityolondon = df[df['Area Name'] == 'City of London']
#     westmin = df[df['Area Name'] == 'Westminster']

#     cityolondon = cityolondon.drop(['Area Code', 'Area Name'], axis=1)
#     westmin = westmin.drop(['Area Code', 'Area Name'], axis=1)

#     cityolondon = cityolondon.reset_index(drop=True)
#     westmin = westmin.reset_index(drop=True)

#     colandwest = cityolondon.add(westmin, fill_value=0)
#     colandwest['Area Code'] = 'E09000001/E09000033'
#     colandwest['Area Name'] = 'City of London/Westminster'

#     df = df.append(colandwest)

#     cornwall = df[df['Area Name'] == 'Cornwall']
#     scilly = df[df['Area Name'] == 'Isles of Scilly']

#     cornwall = cornwall.drop(['Area Code', 'Area Name'], axis=1)
#     scilly = scilly.drop(['Area Code', 'Area Name'], axis=1)

#     cornwall = cornwall.reset_index(drop=True)
#     scilly = scilly.reset_index(drop=True)

#     cornandscilly = cornwall.add(scilly, fill_value=0)
#     cornandscilly['Area Code'] = 'E06000052/E06000053'
#     cornandscilly['Area Name'] = 'Cornwall/Isles of Scilly'

#     df = df.append(cornandscilly)

#     df = df[df['Area Name'].apply(lambda x: x not in ['Westminster', 'City of London', 'Cornwall', 'Isles of Scilly'])]
#     df = df.reset_index(drop=True)

#     return df

### Adding values

In [40]:
topic_combine = {
    'care': {
        "noCare": ["Not a carer"],
        "1to19hoursWeek": ["Yes, 9 hours a week or less", "Yes, 10 to 19 hours a week"],
        "20to49hoursWeek": ["Yes, 20 to 34 hours a week", "Yes, 35 to 49 hours a week"],
        "49PlushoursWeek": ["Yes, 50 or more hours a week"]
    },
    'religion': {
        'Noreligion': ['No religion'],
        'Christian': ['Christian'],
        'Buddhist': ['Buddhist'],
        'Hindu': ['Hindu'],
        'Jewish': ['Jewish'],
        'Muslim': ['Muslim'],
        'Sikh': ['Sikh'],
        'Otherreligion': ['Other religion'],
        'Notstated': ['Not stated']
    },
    'ethnicity': {
        'asian': ['Asian, Asian British or Asian Welsh'],
        'black': ['Black, Black British, Black Welsh, Carribean or African'],
        'mixed': ['Mixed or Multiple ethnic groups'],
        'white': ['White'],
        'other': ['Other ethnic group']
    },
    'health': {
        'good': ['Very good health', 'Good health'],
        'fair': ['Fair health'],
        'bad': ['Bad health', 'Very bad health']
    },
    'economic': {
        'employee': ['Economically active (excluding full-time students): In employment: Employee: Part-time', 'Economically active (excluding full-time students): In employment: Employee: Full-time'],
        'self-employed': ['Economically active (excluding full-time students): In employment: Self-employed: Part-time', 'Economically active (excluding full-time students): In employment: Self-employed: Full-time'],
        'unemployed': ['Economically active: Unemployed: Not a full-time student'],
        'student': ['Economically active: Full-time student'],
        'inactive': ['Economically inactive: Retired', 'Economically inactive (including full-time students): Student', 'Economically inactive: Looking after home or family', 'Economically inactive: Long-term sick or disabled', 'Economically inactive: Other']
    },
    'household': {
        'OnePerson': ['One person household: Aged 66 years and over', 'One person household: Other'],
        '65andOver': ['Single family household: All aged 66 years and over'],
        'Married_Household': ['Single family household: Married or civil partnership couple: No children', 'Single family household: Married or civil partnership couple: Dependent children', 'Single family household: Married or civil partnership couple: All children non-dependent'],
        'Cohabiting': ['Single family household: Cohabiting couple family : No children', 'Single family household: Cohabiting couple family : With dependent children', 'Single family household: Cohabiting couple family : All children non-dependent'],
        'LoneParent': ['Single family household: Lone parent family : With dependent children', 'Single family household: Lone parent family : All children non-dependent'],
        'Other': ['Single family household: Other single family household: Other family composition'],
        'MultipleFamily': ['Multiple-family household: With dependent children', 'Multiple-Family Household : Other, including all full-time students and all aged 66 years and over']
    },
    'marital': {
        'Single': ['Single never married or in a civil partnership'],
        'Married': ['Married or in a registered civil partnership'],
        'Seperated': ['Separated (but still legally married or still legally in a civil partnership)', 'Divorced or civil partnership dissolved'],
        'Widowed': ['Widowed or surviving civil partnership partner']
    },
    'hours': {
        '1-15': ['Part-time: 15 hours or less worked'],
        '16-30': ['Part-time: 16 to 30 hours worked'],
        '31-48': ['Full-time: 31 to 48 hours worked'],
        '49plus': ['Full-time: 49 or more hours worked'],
    },
    'tenure': {
        'owned': ['Owned: Owns outright', 'Owned: Owns with a mortgage or loan'],
        'shared_ownership': ['Shared ownership: Shared ownership'],
        'rented_social': ['Social rented: Rents from council or Local Authority', 'Social rented: Other social rented'],
        'rented_private': ['Private rented: Private landlord or letting agency', 'Private rented: Other private rented'],
        'rent_free': ['Lives rent free']
    },
    'disability': {
        'lot': ['Disabled under the Equality Act: Day-to-day activities limited a lot'],
        'little': ['Disabled under the Equality Act: Day-to-day activities limited a little'],
        'notDisabled': ['Not disabled under the Equality Act :Has long term physical or mental health condition but day-to-day activities are not limited', 'Not disabled under the Equality Act: No long term physical or mental health conditions'],
    },
    'national': {
        'BritishOnly': ['British only identity'],
        'EnglishOnly': ['English only identity'],
        'EnglishBritishOnly': ['English and British only identity'],
        'OtherUK': ['Welsh only identity', 'Welsh and British only identity', 'Scottish only identity', 'Scottish and British only identity', 'Northern Irish only identity', 'Northern Irish and British only identity', 'Cornish only identity', 'Cornish and British only identity', 'Any other combination of UK identities (UK only)', 'Irish only identity', 'Irish and at least one UK identity', 'Other identity and at least one UK identity'],
        'Othernon-UK': ['Other identity only']
    },
    'welsh': {
        'SpeaksWelsh': ['Speaks Welsh'],
        'NoWelsh': ["Doesn't speak Welsh"]
    },
    'population': {
        'Population': ['Population']
    },
    'density': {
        'density': ['Density']
    },
    'age10yr':{
        '0-9': ['0-9'],
        '10-19': ['10-19'],
        '20-29': ['20-29'],
        '30-39': ['30-39'],
        '40-49': ['40-49'],
        '50-59': ['50-59'],
        '60-69': ['60-69'],
        '70-79': ['70-79'],
        '80plus': ['80plus']
    },
    'agemed': {
        'Median Age': ['Median Age']
    }
}

In [41]:
# Create an object that will hold all the dfs for all las in an indexable way
data_las = {}
data_counties = {}
data_regions = {}
data_countries = {}

for topic in topics:
    for year in years:

        # Read a csv
        df = pd.read_csv("./"+topic+"_"+year+".csv")

        # loop through the variables related to this topic
        for key in topic_combine[topic]:

            series = df[topic_combine[topic][key][0]]
            for col in topic_combine[topic][key][1:]:
                series = series + df[col]
            df[key] = series

            # Create this list to avoid dropping columns where the name hasn't changed
            columns_to_drop = [i for i in topic_combine[topic][key] if i not in list(topic_combine[topic].keys())]
            df = df.drop(columns=columns_to_drop)

        # columns_renamed = {i: i.replace(" ", "") for i in df.columns}
        # columns_renamed['Area Code'] = 'Area Code'
        # columns_renamed['Area Name'] = 'Area Name'
                
        # df.rename(columns=columns_renamed, inplace=True)

        # For some reason some areas have an astrix in the title
        # df['Area Name'] = df['Area Name'].str.replace("*", "")
        df['Area Name'] = df['Area Name'].str.replace("\(Met County\)", "").str.replace("()", "").str.strip() 

        # # Merge data for small areas
        # df = merge_small(df)

        # Create a df including only LAs
        data_las[topic+"_"+year] = df[df['Area Name'].apply(lambda x: x in las.keys())]

        # Create a df including only counties
        data_counties[topic+"_"+year] = df[df['Area Name'].apply(lambda x: x in counties.keys())]

        # Reformat Captialised textfor regions and countries
        df['Area Name'] = df['Area Name'].apply(lambda x: x.title().replace(" And ", " and ").replace(" Of ", " of "))

        # Create df for regions
        data_regions[topic+"_"+year] = df[df['Area Name'].apply(lambda x: x in regions.keys())]

        # Create df for regions
        data_countries[topic+"_"+year] = df[df['Area Name'].apply(lambda x: x in countries.keys())]

In [42]:
# These variables hold the relevant data tables and the objects that will be populated with the data, indexible by 
geog_tables = {'las': data_las, 'counties': data_counties, 'regions': data_regions, 'countries': data_countries}
geog_objects = {'las': las, 'counties': counties, 'regions': regions, 'countries': countries}

In [43]:
def find_data(name, geog):

    data = {}

    for topic in topics:

        # Create object for this topic
        data[topic] = {}
        data[topic]['value'] = {}
        data[topic]['perc'] = {}

        for year in years:

            # Find table corresponding to topic and year
            table = geog_tables[geog][topic+'_'+year]

            row = table[table['Area Name'] == name].iloc[0]

            # Find the row of the table relating to the area of iteration
            row = table[table['Area Name'] == name].iloc[0]

            # Transform row into dictionary
            row = row.to_dict()

            # Delete unneeded values
            del row['Area Code']
            del row['Area Name']

            # Translate numpy int into python int for json serialisation
            row = {i: int(row[i]) for i in row}

            # Assign to main data object
            data[topic]['value'][year] = row

            # Calculate percentages: this calcualtion assumes that each figure combines to make a total
            perc = {i: float(round(row[i]/sum(row.values()), 5)) for i in row}
            data[topic]['perc'][year] = perc

        for value_perc in ['value', 'perc']:
            # Calculate percentage point changes
            ob2 = data[topic][value_perc]['2021']
            ob1 = data[topic][value_perc]['2011']
            percob = {i: float(round(ob2[i] - ob1[i], 5)) for i in ob2}
            data[topic][value_perc]['change'] = percob

    return data

In [44]:
# ADDING VALUE AND PERC DATA FOR EACH TOPIC AND YEAR

for name in las.keys():
    las[name]['data'] = find_data(name, 'las')

for name in counties.keys():
    counties[name]['data'] = find_data(name, 'counties')

for name in regions.keys():
     regions[name]['data'] = find_data(name, 'regions')

for name in countries.keys():
     countries[name]['data'] = find_data(name, 'countries')

### Adding ranks

In [45]:
# A function that fixes false neg/pos middle values
def reverse_rank(x, sample):
    half_sample = sample/2
    if (abs(x) > half_sample):
        x = ( ( (np.sign(x)*-1) * (sample + 1)) + (x) )
    return x

reverse_rank(-156, 331)

-156

In [46]:
# This function takes in a raw index and outputs a rank
def process_rank(raw_rank, group_size):
    rank = raw_rank+1

    rank = reverse_rank(rank, group_size)

    return int(rank)

def create_ranks(name, topic, geog, hier, value_perc):
    places = geog_objects[geog]
    ob = {}
    for type in ['2011', '2021', 'change']:
        ob[type] = {}
        for var in places[name]['data'][topic][value_perc][type].keys():

            # Find the value of the data that is being ranked
            this_data = places[name]['data'][topic][value_perc][type][var]

            # Create list of values. If regional rank then filter by areas that share the same parent or country
            if hier == 'regional':
                group_data = [places[i]['data'][topic][value_perc][type][var] for i in places if places[i]['parent']['name'] == places[name]['parent']['name']]
            else:
                group_data = [places[i]['data'][topic][value_perc][type][var] for i in places if places[i]['country'] == places[name]['country']]
            group_size = len(group_data)

            # Sort the data then find the index of the data point we are ranking
            group_data.sort()
            raw_rank = group_data.index(this_data)

            # Process the index into a rank and add to the object that will be outputted
            ob[type][var] = process_rank(raw_rank, group_size)
            
    return ob

In [47]:
# ADDING RANKS FOR EACH TOPIC AND YEAR - NOT NEEDED FOR REGIONS AND COUNTRIES

for name in las.keys():
    for topic in topics:
        las[name]['data'][topic]['perc_rank'] = create_ranks(name, topic, 'las', 'national', 'perc')
        las[name]['data'][topic]['perc_rank_local'] = create_ranks(name, topic, 'las', 'regional', 'perc')
        las[name]['data'][topic]['value_rank'] = create_ranks(name, topic, 'las', 'national', 'value')
        las[name]['data'][topic]['value_rank_local'] = create_ranks(name, topic, 'las', 'regional', 'value')


for name in counties.keys():
    for topic in topics:
        counties[name]['data'][topic]['perc_rank'] = create_ranks(name, topic, 'counties', 'national', 'perc')
        counties[name]['data'][topic]['perc_rank_local'] = create_ranks(name, topic, 'counties', 'regional', 'perc')
        counties[name]['data'][topic]['value_rank'] = create_ranks(name, topic, 'counties', 'national', 'value')
        counties[name]['data'][topic]['value_rank_local'] = create_ranks(name, topic, 'counties', 'regional', 'value')

In [48]:
# A function that finds the value for a variable in the areas ranked above and below this area
def above_below_data(geog, place_name, topic, type):
    place = geog[place_name]
    ob = place['data'][topic][type]
    parent = place['parent']['name']
    country = place['country']
    value_perc = type.split("_")[0]
    if 'local' in type:
        sample = len([i for i in geog if (geog[i]['parent']['name'] == parent)])
    else:
        sample = len([i for i in geog if (geog[i]['country'] == country)])

    above_below = {}

    variables = ob['2021'].keys()
    for variable in variables: 
        above_below[variable] = {}
        rank = ob['2021'][variable]

        # Find the rank above above below
        above =  reverse_rank(rank-1, sample)
        below = reverse_rank(rank+1, sample)

        name_above = [i for i in geog if ((geog[i]['parent']['name'] == parent)|('local' not in type)) & (geog[i]['data'][topic][type]['2021'][variable]==above)]
        if name_above:
            name_above = name_above[0]
            area_above = {'name': name_above, 'value': geog[name_above]['data'][topic][value_perc]['2021'][variable]}
            above_below[variable]['above'] = area_above


        name_below = [i for i in geog if ((geog[i]['parent']['name'] == parent)|('local' not in type)) & (geog[i]['data'][topic][type]['2021'][variable]==below)]
        if name_below:
            name_below = name_below[0]
            area_below = {'name': name_below, 'value': geog[name_below]['data'][topic][value_perc]['2021'][variable]}
            above_below[variable]['below'] = area_below

    ob['above_below'] = above_below
    return ob

# above_below_data(las, 'Gateshead', 'care', type)

In [49]:
# Finds the areas that had a higher proportion but now have a lower proportion for some variable
def overtake_data(geog, place_name, topic, type):
    place = geog[place_name]
    ob = place['data'][topic][type]
    parent = place['parent']['name']
    country = place['country']
    value_perc = type.split("_")[0]

    # Create a list of places that are included in our sample:
    if 'local' in type:
        sample = [i for i in geog if (geog[i]['parent']['name'] == parent)]
    else:
        sample = [i for i in geog if (geog[i]['country'] == country)]

    overtake = {}

    variables = ob['2021'].keys()
    for variable in variables: 

        # Find the values for 2011 and 2021
        value_2011 =  place['data'][topic][value_perc]['2011'][variable]
        value_2021 =  place['data'][topic][value_perc]['2021'][variable]

        # Create a list of places that had higher proportion in 2011
        higher_2011 = [name for name in sample if (geog[name]['data'][topic][value_perc]['2011'][variable] > value_2011) ]

        # Filter that list to only include places that were lower in 2021
        overtaken = [name for name in higher_2011 if (geog[name]['data'][topic][value_perc]['2021'][variable] < value_2021) ]

        overtake[variable] = overtaken

    ob['overtake'] = overtake
    return ob
# overtake_data(las, 'Darlington', 'care', 'perc_rank_local')

In [50]:
# Finds the highest and lowest scoring three areas for some variable
def top_bottom_data(geog, place_name, topic, type):
    place = geog[place_name]
    ob = place['data'][topic][type]
    parent = place['parent']['name']
    country = place['country']
    value_perc = type.split("_")[0]

    # Create a list of places that are included in our sample:
    if 'local' in type:
        sample = [i for i in geog if (geog[i]['parent']['name'] == parent)]
    else:
        sample = [i for i in geog if (geog[i]['country'] == country)]

    top_bottom = {}
    top_bottom_11 = {}

    variables = ob['2021'].keys()
    for variable in variables:

        t_b_var = {}
        t_b_var_11 = {}

        for rank in [1,2,3,-3,-2,-1]:
            name = [name for name in sample if geog[name]['data'][topic][type]['2021'][variable] == rank]
            if name:
                name = name[0]
                # Find the values for 2011 and 2021
                value_2011 =  geog[name]['data'][topic][value_perc]['2011'][variable]
                value_2021 =  geog[name]['data'][topic][value_perc]['2021'][variable]
                t_b_var[rank] = {'2011': value_2011, '2021': value_2021, 'name': name, 'change': value_2021-value_2011}

            # Repeat for 2011 ranks
            name = [name for name in sample if geog[name]['data'][topic][type]['2011'][variable] == rank]
            if name:
                name = name[0]
                # Find the values for 2011 and 2021
                value_2011 =  geog[name]['data'][topic][value_perc]['2011'][variable]
                value_2021 =  geog[name]['data'][topic][value_perc]['2021'][variable]
                t_b_var_11[rank] = {'2011': value_2011, '2021': value_2021, 'name': name, 'change': round(value_2021-value_2011, 5)}

        top_bottom[variable] = t_b_var
        top_bottom_11[variable] = t_b_var_11

    ob['top_bottom'] = top_bottom
    ob['top_bottom_11'] = top_bottom_11
    return ob

In [51]:
# Add the additional information about associated ranking areas
for place in las:
    for topic in topics:
        for type in ['perc_rank', 'perc_rank_local']:
            las[place]['data'][topic][type] = above_below_data(las, place, topic, type)
            las[place]['data'][topic][type] = overtake_data(las, place, topic, type)
            las[place]['data'][topic][type] = top_bottom_data(las, place, topic, type)

#### Find each LA and county's near and similar areas

In [52]:
# This is done after first iteration to point to other objects in dictionary
for i in tiers.index:

    # Create a variable which is an iterated row of the df
    row = tiers.loc[i]

    # The name of the area attached to this row
    nameLA = row['LAD21NM']
    nameCounty = row['CTY21NM']

    # print(row['LAD21NM'])
    # print(row['LAD21CD'])

    # las[nameLA]['near'] = lads_neigh[row['LAD21CD']]
    las[nameLA]['near'] = [las[i] for i in las if las[i]['code'] == lads_neigh[row['LAD21CD']]][0].copy()
    # las[nameLA]['near']['near'] = 'confudsed'
    las[nameLA]['near']['near'] = ""
    del las[nameLA]['near']['near']
    las[nameLA]['near']['similar'] = ""
    del las[nameLA]['near']['similar']

    # las[nameLA]['similar'] = get_similar(row['LAD21CD'])
    if get_similar(row['LAD21CD']):
        las[nameLA]['similar'] = [las[i] for i in las if las[i]['code'] == get_similar(row['LAD21CD'])][0].copy()
        las[nameLA]['similar']['near'] = ""
        del las[nameLA]['similar']['near']
        las[nameLA]['similar']['similar'] = ""
        del las[nameLA]['similar']['similar']

    # counties[nameCounty]['near'] = county_neigh[row['CTY21CD']] 
    counties[nameCounty]['near'] = [counties[i] for i in counties if counties[i]['code'] == county_neigh[row['CTY21CD']]][0].copy()
    counties[nameCounty]['near']['near'] = ""
    del counties[nameCounty]['near']['near']
    counties[nameCounty]['near']['similar'] = ""
    del counties[nameCounty]['near']['similar']

In [53]:
# Add a piece of data which is used in the age band section

for geography in [counties, las]:
    for area_name in geography:
        area = geography[area_name]
        ageBandChange = sorted([(i, area['data']['age10yr']['value']['2021'][i]-area['data']['age10yr']['value']['2011'][i]) for i in area['data']['age10yr']['value']['2011'].keys()], reverse=True, key=lambda x: x[1])

        # ageBandChange= [i for i in ageBandChange if i[0]!='all']
        ageBandPos = sorted([i for i in ageBandChange if i[1]>0], reverse=True, key=lambda x: abs(x[1]))
        ageBandNeg = sorted([i for i in ageBandChange if i[1]<0], reverse=True, key=lambda x: abs(x[1]))

        area['data']['age10yr']['absChange'] = {}
        area['data']['age10yr']['absChange']['pos'] = ageBandPos
        area['data']['age10yr']['absChange']['neg'] = ageBandNeg

# Below is the story finding section

In [54]:
excluded_variables = ["NoWelsh", "16-30", "31-48", "Widowed", "65andOver", "Other", "MultiOther", "fair", "rent_free", "shared_ownership",  "female", "male", "inactive", '1to19hoursWeek', "Religionnotstated", "OtherUK", "Married_Household"]

# Add a stories array for each area
for name in las:
    place = las[name]
    stories = []
    for a in place['data'].keys():
        for d in place['data'][a]['perc']['change'].keys():
            if d not in excluded_variables:
                if a in ['population', 'agemed']:
                    b = 'value'
                else:
                    b = 'perc'
                stories.append({
                    'label': a+'_'+b+'_change'+'_'+d, 
                    'locRank': place['data'][a][b+'_rank_local']['change'][d], 
                    'natRank': place['data'][a][b+'_rank']['change'][d], 
                    '2011': place['data'][a][b]['2011'][d],
                    '2021': place['data'][a][b]['2021'][d],
                    'change': place['data'][a][b]['change'][d],
                    'perc_cha': 100*(place['data'][a][b]['change'][d]/place['data'][a][b]['2011'][d])
                })
    place['stories'] = stories

In [55]:
# A function which outputs 1.0 for any positive number and -1.0 for any negative number
sign = lambda x: math.copysign(1, x)
sign(-9.7)

# A function which takes in a list of stories and combines the 'type' lists of any repeated variables and gets rid of the duplcate
def combinestories(stories):
    stories2 = []
    for s in stories:
        if s['label'] in [i['label'] for i in stories2]:
            index = next((i for i, item in enumerate(stories2) if item['label'] == s['label']), -1)
            stories2[index]['type'] = stories2[index]['type'] + s['type']
        else:
            stories2.append(s)

    return stories2

# Functions that find regional and national equivelent to a local data point
def reg(story):
    s=story['label'].split("_")
    if len(s)>4:
        s[3] = s[3] + "_" + s[4]
    return region['data'][s[0]][s[1]][s[2]][s[3]]

def cou(i):
    s=i['label'].split("_")
    if len(s)>4:
        s[3] = s[3] + "_" + s[4]
    return country['data'][s[0]][s[1]][s[2]][s[3]]

def simi(i):
    s=i['label'].split("_")
    if len(s)>4:
        s[3] = s[3] + "_" + s[4]
    return similar['data'][s[0]][s[1]][s[2]][s[3]]

def near(i):
    s=i['label'].split("_")
    if len(s)>4:
        s[3] = s[3] + "_" + s[4]
    return nearby['data'][s[0]][s[1]][s[2]][s[3]]

In [56]:
for name in las:
    place = las[name]
    region = regions[place['parent']['name']]
    country = countries[place['country']]
    # Skip similar-area based stories if this LA doesn't have a similar area
    if 'similar' in place:
        similar = place['similar']
    nearby = place['near']

    # Create a list of objects ordered by the size of difference between regional and local change
    regDiff = [(i, {'otherchange': reg(i), 'dif': i['change']-reg(i)}) for i in place['stories']]
    couDiff = [(i, {'otherchange': cou(i), 'dif': i['change']-cou(i)}) for i in place['stories']]
    if 'similar' in place:
        simiDiff = [(i, {'otherchange': simi(i), 'dif': i['change']-simi(i)}) for i in place['stories']]
    nearDiff = [(i, {'otherchange': near(i), 'dif': i['change']-near(i)}) for i in place['stories']]

    # Create list of objects ordered by size of PP change
    vallist = sorted(place['stories'], reverse=True, key=lambda x: x['change'])

    # Iterate through vallist to find the largest percentage point increase for ethnicity and religion
    ethStory = next(filter(lambda x: x['label'].split("_")[0] == 'ethnicity', vallist))
    relStory = next(filter(lambda x: x['label'].split("_")[0] == 'religion', vallist))

    # Add the story type descriptor for these two stories
    ethStory = [{**ethStory, **{'type':['ethrel']}}]
    relStory = [{**relStory, **{'type':['ethrel']}}]

    # Remove all religion and ethnicity variables from our original story list and add the two stories pre-selected
    place['stories'] = [i for i in place['stories'] if (i['label'].split("_")[0]!='religion') & (i['label'].split("_")[0]!='ethnicity')]+ethStory+relStory

    # This records the story type for each variable in our stories list
    stories = []
    if (country['name']=='Wales'):
        stories=stories+[{**i, **{'type':['welsh']}} for i in place['stories'] if i['label'] == 'welsh_perc_change_Speaks Welsh']
    stories+[{**i, **{'type':['pop']}} for i in place['stories'] if i['label'] == 'population_value_change_Population']
    stories=stories+[{**i, **{'type':['size']}} for i in place['stories'] if ((abs(i['change'])/abs(i['2011'])>0.2)&(abs(i['change']) > 1))]
    stories=stories+[{**i, **{'type':['locRank']}} for i in place['stories'] if (abs(i['locRank']) < 4)&(abs(i['change']) > 0.5)]
    stories=stories+[{**i, **{'type':['natRank']}} for i in place['stories'] if (abs(i['natRank']) < 4)&(abs(i['change']) > 0.5)]
    stories=stories+[{**i[0], **{'type':['couBuck']}} for i in couDiff if ((abs(i[1]['otherchange'])>1) & (abs(i[0]['change'])>1) & (sign(i[1]['otherchange'])!=sign(i[0]['change'])))]
    stories=stories+[{**i[0], **{'type':['regBuck']}} for i in regDiff if ((abs(i[1]['otherchange'])>1) & (abs(i[0]['change'])>1) & (sign(i[1]['otherchange'])!=sign(i[0]['change'])))]
    stories=stories+[{**i[0], **{'type':['couDiff']}} for i in couDiff if ((abs(i[1]['dif'])>2) & (sign(i[1]['otherchange'])==sign(i[0]['change'])))]
    stories=stories+[{**i[0], **{'type':['regDiff']}} for i in regDiff if ((abs(i[1]['dif'])>2) & (sign(i[1]['otherchange'])==sign(i[0]['change'])))]
    stories=stories+[{**i[0], **{'type':['nearDiff']}} for i in nearDiff if ((abs(i[1]['dif'])>2) & (sign(i[1]['otherchange'])==sign(i[0]['change'])))]
    if 'similar' in place:
        stories=stories+[{**i[0], **{'type':['simiDiff']}} for i in simiDiff if ((abs(i[1]['dif'])>2) & (sign(i[1]['otherchange'])==sign(i[0]['change'])))]
    stories=stories+[{**i, **{'type':['general']}} for i in place['stories']]
    stories = combinestories(stories)

    # This section puts stories in order based on how notable each variable is
    stories = sorted(stories, reverse=True, key=lambda x: abs(x['locRank'])<4)
    stories = sorted(stories, reverse=True, key=lambda x: abs(x['natRank'])<4)
    stories = sorted(stories, reverse=True, key=lambda x: abs(x['locRank'])<3)
    stories = sorted(stories, reverse=True, key=lambda x: abs(x['natRank'])<3)
    stories = sorted(stories, reverse=True, key=lambda x: x['type']=='regBuck')
    stories = sorted(stories, reverse=True, key=lambda x: x['type']=='couBuck')
    stories = sorted(stories, reverse=True, key=lambda x: x['label']=='welsh_perc_change_Speaks Welsh')
    stories = sorted(stories, reverse=True, key=lambda x: x['label']=='agemed_value_change_Median Age')
    stories = sorted(stories, reverse=True, key=lambda x: x['label']=='population_value_change_Population')

    # Remove stories from topics that have appeared higher up in the story list
    stories_uniquetopic = []
    dontInc = ['density', 'age10yr']
    for story in stories:
        if story['label'].split("_")[0] not in dontInc:
            stories_uniquetopic.append(story)
            dontInc.append(story['label'].split("_")[0])

    place['stories'] = stories_uniquetopic

In [57]:
regions['North West']

{'name': 'North West',
 'parent': {'name': 'England', 'code': 'E92000001'},
 'country': 'England',
 'code': 'E12000002',
 'type': 'Region',
 'data': {'care': {'value': {'2011': {'noCare': 40411,
     '1to19hoursWeek': 92771,
     '20to49hoursWeek': 91479,
     '49PlushoursWeek': 24883},
    '2021': {'noCare': 63342,
     '1to19hoursWeek': 65543,
     '20to49hoursWeek': 113458,
     '49PlushoursWeek': 36744},
    'change': {'noCare': 22931.0,
     '1to19hoursWeek': -27228.0,
     '20to49hoursWeek': 21979.0,
     '49PlushoursWeek': 11861.0}},
   'perc': {'2011': {'noCare': 0.16194,
     '1to19hoursWeek': 0.37176,
     '20to49hoursWeek': 0.36658,
     '49PlushoursWeek': 0.09971},
    '2021': {'noCare': 0.22696,
     '1to19hoursWeek': 0.23485,
     '20to49hoursWeek': 0.40653,
     '49PlushoursWeek': 0.13166},
    'change': {'noCare': 0.06502,
     '1to19hoursWeek': -0.13691,
     '20to49hoursWeek': 0.03995,
     '49PlushoursWeek': 0.03195}}},
  'religion': {'value': {'2011': {'Christian': 

In [58]:
# OUTPUT DATA
for region in regions:
    try:
        with open('../static/data/json_new/place/'+regions[region]['code']+'.json', 'w') as outfile:
            json.dump(regions[region], outfile)
    except:
        print("Failed: ", regions[region]['name'])

In [59]:
for country in countries:
    try:
        with open('../static/data/json_new/place/'+countries[country]['code']+'.json', 'w') as outfile:
            json.dump(countries[country], outfile)
    except:
        print("Failed: ", countries[country]['name'])

In [60]:
for la in las:
    try:
        with open('../static/data/json_new/place/'+las[la]['code']+'.json', 'w') as outfile:
            json.dump(las[la], outfile)
    except:
        print("Failed: ", las[la]['name'])